In [ ]:
using HDF5
using Dates 
using Images
#https://portal.hdfgroup.org/display/HDF5/Introduction+to+HDF5
# https://docs.julialang.org/en/v1/stdlib/Dates/#Dates.format-Tuple{TimeType,AbstractString} 
# https://cpb-us-w2.wpmucdn.com/sites.wustl.edu/dist/f/1861/files/2019/02/02-imaris-image-properties-and-edit-2jj1avj.pdf

In [343]:
# https://github.com/tlambert03/imarispy/blob/master/imarispy/imaris.py
function  array_to_ims(array, fname::String="myfile.ims",
              subsamp=((1, 1, 1)),
              chunks=((16, 128, 128)),
              compression=2,
              thumbsize=256,
              dx=0.108, dz=0.5, dt_min=10)

    df = DateFormat("Y-mm-dd HH:MM:SS.sss")
    RecordingDate = DateTime(2020,5,7,14,19,22,0)
    
    nx, ny, nz, nt = size(array)
    nc = 1
    nr = 1     #nr = length(subsamp)
    
    GROUPS = [
        "DataSetInfo",
        "Thumbnail",
        "DataSetTimes",
        "DataSetInfo/Imaris",
        "DataSetInfo/Image",
        "DataSetInfo/TimeInfo"
    ]


    ATTRS = [
        ("/", "ImarisDataSet", "ImarisDataSet"),
        ("/", "ImarisVersion", "5.5.0"),
        ("/", "DataSetInfoDirectoryName", "DataSetInfo"),
        ("/", "ThumbnailDirectoryName", "Thumbnail"),
        ("/", "DataSetDirectoryName", "DataSet"),
        ("DataSetInfo/Imaris", "Version", "9.5"),
        ("DataSetInfo/Imaris", "ThumbnailMode", "thumbnailMIP"),
        ("DataSetInfo/Imaris", "ThumbnailSize", thumbsize),
        ("DataSetInfo/Image", "X", nx),
        ("DataSetInfo/Image", "Y", ny),
        ("DataSetInfo/Image", "Z", nz),
        ("DataSetInfo/Image", "NumberOfChannels", nc),
        ("DataSetInfo/Image", "Noc", nc),
        ("DataSetInfo/Image", "Unit", "um"),
        ("DataSetInfo/Image", "Description", "Hi"),
        ("DataSetInfo/Image", "MicroscopeModality", "Inverted Microscope"),
        ("DataSetInfo/Image", "RecordingDate", Dates.format(RecordingDate, df)),
        ("DataSetInfo/Image", "Name", "qblab"),
        ("DataSetInfo/Image", "ExtMin0", "0"),
        ("DataSetInfo/Image", "ExtMin1", "0"),
        ("DataSetInfo/Image", "ExtMin2", "0"),
        ("DataSetInfo/Image", "ExtMax0", nx * dx),
        ("DataSetInfo/Image", "ExtMax1", ny * dx),
        ("DataSetInfo/Image", "ExtMax2", nz * dz),
        ("DataSetInfo/Image", "LensPower", "40x"),
        ("DataSetInfo/TimeInfo", "DatasetTimePoints", nt),
        ("DataSetInfo/TimeInfo", "FileTimePoints", nt)
    ];
    
    COLORS = ("1 1 1", "1 0 1", "1 1 0", "0 0 1")
    for c in 0:nc-1
        grp = "DataSetInfo/Channel $c"
        push!(GROUPS, grp)
        push!(ATTRS, (grp, "ColorOpacity", 1))
        push!(ATTRS, (grp, "ColorMode", "BaseColor"))
        push!(ATTRS, (grp, "Color", COLORS[1]))
        push!(ATTRS, (grp, "GammaCorrection", 1))
        push!(ATTRS, (grp, "ColorRange", "0 255"))
        push!(ATTRS, (grp, "Name", "Channel $c"))
    end
    
    for t in 0:nt-1
        strr = Dates.format(RecordingDate + Dates.Minute(dt_min*t), df)
        #strr = "2020-05-07 {:02d}:{:02d}:{:02d}.000".format(h, m, s)
        push!(ATTRS, ("DataSetInfo/TimeInfo", "TimePoint$t", strr))
        #TODO: Imaris don't use these timestamps as time axis
    end
    
    h5open(fname, "w") do file
         for grp in GROUPS
            g_create(file, grp)
        end
        
        for info in ATTRS
            grp, key = info[1:2]
            value = "$(info[3])"
            h5a_write_S1(file[grp], key, value)
        end

        file["Thumbnail/Data"] = UInt8.(maximum(array[:,:, 1:20,1], dims=3).>>8)
        
        # add data
        for t in 0:nt-1
            for c in 0:nc-1
                data = array[:, :, :, t+1]
                for r in 0:nr-1
                    edges, count = build_histogram(data, 256)
                    grp = g_create(file, "/DataSet/ResolutionLevel $r/TimePoint $t/Channel $c/")
                    grp["Histogram"] = UInt64.(count[1:end])
                    h5a_write_S1(grp, "HistogramMin", "1000")
                    h5a_write_S1(grp, "HistogramMax", "$(edges[end])")
                    grp["Data", "chunk", (256,256,4), "compress", compression] = data
                    h5a_write_S1(grp, "ImageSizeX", "$(size(data)[1])")
                    h5a_write_S1(grp, "ImageSizeY", "$(size(data)[2])")
                    h5a_write_S1(grp, "ImageSizeZ", "$(size(data)[3])")
                end
            end
        end
    end
end

array_to_ims (generic function with 9 methods)

In [250]:
function h5a_write_S1(parent, key, value)
    dspace = dataspace((length(value),))
    try
        attr = a_create(parent, key,  HDF5Datatype(HDF5.H5T_C_S1), dspace)
        HDF5.writearray(attr, HDF5.H5T_C_S1, value)
    finally
        close(dspace)
    end
    nothing
end

h5a_write_S1 (generic function with 1 method)

In [264]:
img = load(File(format"TIFF","d16s1_1_otsu_norm.ome.tiff"));

┌ Info: Precompiling ImageMagick [6218d12a-5da1-5696-b52f-db25d2ecc6d1]
└ @ Base loading.jl:1260


In [267]:
x = reshape(img, (512,512, 20, 138));

In [286]:
test_img = rand(UInt16, (512, 512, 24, 138));

In [317]:
reinterpret.(N0f8.(x));

In [335]:
@time array_to_ims(test_img, "test.ims")

 10.019962 seconds (16.66 k allocations: 1.619 GiB, 1.10% gc time)


In [344]:
@time array_to_ims(reinterpret.(real(x)), "d16s1_1_otsu_norm.ome.tiff.ims")

 21.177768 seconds (793.12 k allocations: 4.092 GiB, 1.47% gc time)


In [35]:
y = h5readattr("d16s1_2test.h5", "DataSetInfo/Channel 0")

Dict{String,Array{String,1}} with 9 entries:
  "SamplesPerPixel" => ["1"]
  "Description"     => ["(", "d", "e", "s", "c", "r", "i", "p", "t", "i"  …  "s…
  "ColorRange"      => ["0", ".", "0", "0", "0", " ", "8", "1", "1", ".", "0", …
  "GammaCorrection" => ["1", ".", "0", "0", "0"]
  "ColorOpacity"    => ["1", ".", "0", "0", "0"]
  "ColorMode"       => ["B", "a", "s", "e", "C", "o", "l", "o", "r"]
  "Name"            => ["(", "n", "a", "m", "e", " ", "n", "o", "t", " ", "s", …
  "Color"           => ["1", ".", "0", "0", "0", " ", "0", ".", "0", "0", "0", …
  "ID"              => ["C", "h", "a", "n", "n", "e", "l", ":", "0", ":", "0"]

In [256]:
z = h5readattr("test.h5", "DataSetInfo/Channel 0")

Dict{String,Array{String,1}} with 6 entries:
  "ColorRange"      => ["0", " ", "2", "5", "5"]
  "GammaCorrection" => ["1"]
  "ColorOpacity"    => ["1"]
  "ColorMode"       => ["B", "a", "s", "e", "C", "o", "l", "o", "r"]
  "Name"            => ["C", "h", "a", "n", "n", "e", "l", " ", "0"]
  "Color"           => ["1", " ", "1", " ", "1"]